<a href="https://colab.research.google.com/github/TurboPug2101/NLP/blob/main/FakeNewsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

In [3]:
import pandas as pd

In [10]:
df=pd.read_csv('train.csv',error_bad_lines=False, engine="python")


<ipython-input-10-78b229503774>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',error_bad_lines=False, engine="python")
Skipping line 1259: unexpected end of data


In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
df.isnull().sum()

id          0
title      36
author    132
text        3
label       0
dtype: int64

In [13]:
df.shape

(1257, 5)

In [14]:
df=df.dropna()

In [15]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [17]:
#independent feature
X=df.drop('label',axis=1)

In [18]:
#get dependent feature
y=df['label']

In [22]:
X['title'].shape

(1089,)

In [23]:
y.shape

(1089,)

In [24]:
import tensorflow as tf


In [25]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [26]:
voc_size=2000

One Hot representation

In [27]:
messages=X.copy()

In [28]:
messages.reset_index(inplace=True)

In [29]:
messages['title'][6]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [30]:
import nltk
import re
from nltk.corpus import stopwords

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
##Pre Processing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
    try:
        review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
        review=review.lower()
        review=review.split()
        review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
        review=' '.join(review)
        corpus.append(review)
    except KeyError:
        print(f"KeyError: Unable to find key {i} in the 'messages' DataFrame.")


In [36]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [40]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[1]

[1488, 1812, 1001, 248, 1929, 1429, 1134]

Embedding Layer

In [41]:
sent_len=20;
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 1691  218  522]
 [   0    0    0 ... 1929 1429 1134]
 [   0    0    0 ... 1100  291  850]
 ...
 [   0    0    0 ...   25 1826  582]
 [   0    0    0 ...   40  857  757]
 [   0    0    0 ...  932  290  921]]


In [43]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1488, 1812, 1001,  248, 1929, 1429, 1134], dtype=int32)

each word here given by index (eg 1488 ) will be conv to vector of 40 features

Creating Model

In [45]:
emb_vector_feat=40
model=Sequential()
model.add(Embedding(voc_size,emb_vector_feat,input_length=sent_len))
model.add(LSTM(100))
#100 neurons
model.add(Dense(1,activation='sigmoid'))
#binary output
model.compile( loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            80000     
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 136,501
Trainable params: 136,501
Non-trainable params: 0
_________________________________________________________________
None


emb layer 20 inp ,40 out
lstm layer 100 neuron

In [46]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [47]:
X_final.shape,y_final.shape

((1089, 20), (1089,))

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X_final,y_final,test_size=0.33,random_state=42)

MODEL TRAINING

In [49]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
12/12 [==============================] - 11s 298ms/step - loss: 0.6834 - accuracy: 0.5501 - val_loss: 0.6700 - val_accuracy: 0.5528
Epoch 2/10
12/12 [==============================] - 2s 132ms/step - loss: 0.6382 - accuracy: 0.5844 - val_loss: 0.6029 - val_accuracy: 0.8333
Epoch 3/10
12/12 [==============================] - 1s 114ms/step - loss: 0.5312 - accuracy: 0.8066 - val_loss: 0.5364 - val_accuracy: 0.8389
Epoch 4/10
12/12 [==============================] - 1s 114ms/step - loss: 0.3679 - accuracy: 0.8889 - val_loss: 0.4785 - val_accuracy: 0.8444
Epoch 5/10
12/12 [==============================] - 1s 80ms/step - loss: 0.2255 - accuracy: 0.9328 - val_loss: 0.4725 - val_accuracy: 0.8194
Epoch 6/10
12/12 [==============================] - 2s 165ms/step - loss: 0.1352 - accuracy: 0.9561 - val_loss: 0.4285 - val_accuracy: 0.8417
Epoch 7/10
12/12 [==============================] - 1s 44ms/step - loss: 0.0917 - accuracy: 0.9684 - val_loss: 0.4603 - val_accuracy: 0.8333
Epoch 8

Performance Matrix

In [50]:
y_pred=model.predict(X_test)

12/12 [==============================] - 0s 2ms/step


In [51]:
y_pred=np.where(y_pred>0.5,1 ,0)
#1 => fake if prob >0.5

In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
confusion_matrix(y_test,y_pred)

array([[170,  29],
       [ 32, 129]])

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8305555555555556

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85       199
           1       0.82      0.80      0.81       161

    accuracy                           0.83       360
   macro avg       0.83      0.83      0.83       360
weighted avg       0.83      0.83      0.83       360

